## Construir una aplicación utilizando un almacén de datos clave-valor NoSQL

En este tutorial, crearás una aplicación de librería que muestre un catálogo de productos. Los productos suelen contener identificadores únicos y atributos como descripciones, cantidades, ubicaciones y precios. El método para recuperar este tipo de atributos (en concreto, el patrón de acceso) suele ser una búsqueda clave-valor basada en el identificador único del producto. Esto significa que una aplicación puede recuperar estos otros atributos cuando se proporciona el identificador único de un producto.

Si bien el catálogo de productos puede comenzar con unos pocos productos, debe tener la capacidad de escalar a miles de millones si es necesario sin tener que cambiar su arquitectura o requerir una base de datos diferente. También debe mantener un desempeño rápido y predecible a cualquier escala para estas búsquedas de valores clave. Teniendo en cuenta estos requisitos, Amazon DynamoDB es una buena opción como sistema de registro duradero para la librería, ya que ofrece un desempeño de baja latencia y se escala a medida que crece la aplicación. Otra ventaja es que no necesita administrar ningún servidor ni clúster.

![image.png](https://d1.awsstatic.com/nosql-key-value-data-store/Initial%20Visualization.1944529d4d62522171b6091ee74bb343bcdd940f.jpg)


### Introducción
Como se mencionó al principio de este tutorial, vas a crear una aplicación de librería online con un amplio catálogo de productos. Cuando un usuario accede a esta aplicación, se mostrarán los libros que están en stock e información básica sobre esos libros, como autor, título, categoría y más.

La aplicación debe ser capaz de recuperar un libro concreto mediante una combinación de su título y autor para que el usuario pueda obtener información adicional sobre el libro. La aplicación también debe permitir a los usuarios ver todos los libros de una categoría determinada, como historia o biografía, para facilitar el descubrimiento de libros en el catálogo. Estos libros también pueden tener distintos formatos, como tapa dura, rústica o audiolibro. Estos formatos pueden actualizarse con el tiempo.

Terminología y conceptos
Los siguientes conceptos de DynamoDB desempeñan un papel clave en este tutorial:
 
* **Tabla**: Se trata simplemente de una colección de datos. Una tabla es específicamente la estructura en la que DynamoDB almacena estos datos. En nuestro caso, tendremos una tabla books para almacenar nuestro catálogo de productos.

* **Elemento**: Un único registro de datos en una tabla de DynamoDB. Es comparable a una fila en una base de datos relacional. Cada elemento de nuestra tabla corresponderá a un libro de nuestro catálogo de productos.

* **Atributo**: Un único elemento de datos de un elemento. Es comparable a una columna en una base de datos relacional. Sin embargo, a diferencia de las columnas de una base de datos relacional, no es necesario especificar los atributos al crear la tabla, salvo la clave primaria. En nuestro caso, los atributos consistirán en elementos como Título, Autor y Categoría.

* **Clave primaria**: Una clave primaria es un identificador único para un solo elemento en una tabla DynamoDB. Al crear una tabla, debe especificar el nombre y el tipo de la clave primaria. La clave primaria identifica de forma única cada elemento escrito en una tabla. Una clave primaria simple consta de un único atributo, y una clave primaria compuesta consta de dos atributos. Por ejemplo, utilizaremos una combinación de Título y Autor como identificador único para cada libro.

* **PartiQL**: Se trata de un lenguaje de consulta compatible con SQL que le permite codificar sus operaciones de datos de DynamoDB utilizando sintaxis SQL.

### Modelo de datos
Al crear una aplicación, siempre hay que dedicar tiempo a diseñar los modelos de datos necesarios en la lógica de la aplicación. El diseño del modelo de datos debe tener en cuenta las necesidades de acceso a los datos de la aplicación, tanto para la lectura como para la escritura de datos.

DynamoDB es una base de datos no relacional, lo que significa que no es necesario especificar el esquema completo por adelantado al crear una tabla. Solo tiene que declarar la clave principal de la tabla, que identifica de forma exclusiva cada registro de la tabla. Esto reduce el coste inicial del diseño del modelo de datos, ya que se puede modificar fácilmente el esquema a medida que cambian las necesidades de la aplicación.

Como se ha mencionado anteriormente, su aplicación necesita recuperar un libro individual por su título y autor. Dado que la combinación de título y autor es un identificador único de un libro, puede utilizar estos atributos como clave principal de la tabla. Su aplicación también necesita almacenar información sobre la categoría de nuestro libro, como historia o biografía, así como los formatos disponibles de su libro -cubierta dura, tapa blanda o audiolibro- que se asignan a los números de artículo en su sistema de inventario.

### Crear la tabla

In [1]:
import boto3
from botocore.exceptions import ClientError
import pandas as pd
import base64
from spdynamodb import DynamoTable
from datetime import datetime
import json
import time
from decimal import Decimal

In [2]:
#dt = DynamoTable(profile_name='my-profile')
dt=DynamoTable()
try:
    dt.select_table('Bookks')
    print(dt)
except:
    dt.create_table(
        table_name='Bookks',
        partition_key='Author',
        partition_key_type='S',
        sort_key='Title',
        sort_key_type='S',
    )

- Table name: Bookks            
- Table arn: arn:aws:dynamodb:us-east-1:089715336747:table/Bookks            
- Table creation: 2023-05-27 19:49:51            
- [{'AttributeName': 'Author', 'KeyType': 'HASH'}, {'AttributeName': 'Title', 'KeyType': 'RANGE'}]            
- [{'AttributeName': 'Author', 'AttributeType': 'S'}, {'AttributeName': 'Category', 'AttributeType': 'S'}, {'AttributeName': 'Title', 'AttributeType': 'S'}]            
- Point-in-time recovery status: DISABLED  |  Delete protection: False


### Insertar elementos en nuestra tabla

In [10]:
%%writefile partiqlbatch.json
[
    {
        "Author": "John Grisham",
        "Title": "The Rainmaker",
        "Category": "Suspense",
        "Formats": {
            "Hardcover": true,
            "Paperback": true,
            "Audiobook": true
        }
    },
    {
        "Author": "William Shakespeare",
        "Title": "The Tempest",
        "Category": "Romance",
        "Formats": {
            "Hardcover": true,
            "Paperback": true,
            "Audiobook": false
        }
    },
    {
        "Author": "Charles Dickens",
        "Title": "Great Expectations",
        "Category": "Romance",
        "Formats": {
            "Hardcover": false,
            "Paperback": true,
            "Audiobook": false
        }
    },
    {
        "Author": "James Patterson",
        "Title": "Along Came a Spider",
        "Category": "Suspense",
        "Formats": {
            "Hardcover": true,
            "Paperback": true,
            "Audiobook": true
        }
    },
    {
        "Author": "Dr. Seuss",
        "Title": "Green Eggs and Ham",
        "Category": "Children",
        "Formats": {
            "Hardcover": true,
            "Paperback": true,
            "Audiobook": true
        }
    },
    {
        "Author": "Julie Simon",
        "Title": "Learn Amazon SageMaker",
        "Category": "Technology",
        "Formats": {
            "Hardcover": true,
            "Paperback": false,
            "Audiobook": false
        }
    }
]

Overwriting partiqlbatch.json


In [11]:
dt.load_json("partiqlbatch.json")

In [3]:
dt.query(pk_value="Dr. Seuss", sk_value="Green Eggs and Ham", consumed_capacity=True)

Consumed Capacity: 0.5


{'Title': 'Green Eggs and Ham',
 'Formats': {'Hardcover': True, 'Paperback': True, 'Audiobook': True},
 'Author': 'Dr. Seuss',
 'Category': 'Children'}

### Crear índice secundario global

In [5]:
dt.create_global_secondary_index(
    att_name="Category",
    att_type="S",
    i_name="CategoryIndex"
)

status = dt.check_status_gsi()
if status == 'CREATING':
    print("Global secondary index is being created, this may take a few minutes...")
    while status == 'CREATING':
        status = dt.check_status_gsi()
        time.sleep(30)
print("Global secondary index created.")

Global secondary index is being created, this may take a few minutes...
Global secondary index created.


In [19]:
dt.query_partiql('SELECT * FROM Bookks.CategoryIndex WHERE Category = \'Technology\'')

ExecuteStatement executed successfully.


[{'Title': 'Learn Amazon SageMaker',
  'Formats': {'Hardcover': True, 'Paperback': False, 'Audiobook': False},
  'Author': 'Julie Simon',
  'Category': 'Technology'}]

### Update

In [8]:
dt.query_partiql('UPDATE Bookks SET Formats.Audiobook = \'JCV555\' WHERE Author = \'James Patterson\' AND Title = \'Along Came a Spider\'', consumed_capacity='TOTAL')

ExecuteStatement executed successfully.
Consumed Capacity: 1.0
Not found any items


In [10]:
dt.query_partiql('UPDATE Bookks REMOVE Formats.Audiobook WHERE Author = \'James Patterson\' AND Title = \'Along Came a Spider\'', consumed_capacity='TOTAL')

ExecuteStatement executed successfully.
Consumed Capacity: 2.0
Not found any items


In [38]:
dt.table.delete_item(Key={'Author': 'Charles Dickens', 'Title': 'Great Expectations'}, ReturnValues='ALL_OLD', ReturnConsumedCapacity='TOTAL')

{'ConsumedCapacity': {'TableName': 'Bookks', 'CapacityUnits': 1.0},
 'ResponseMetadata': {'RequestId': '7PQRG7N4SFUPA4TV97R0UJHQSRVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 28 May 2023 00:27:00 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '63',
   'connection': 'keep-alive',
   'x-amzn-requestid': '7PQRG7N4SFUPA4TV97R0UJHQSRVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '3447348954'},
  'RetryAttempts': 0}}